# installing the ollama3 of 1 billion paramater model
<!-- !ollama run llama3.2:1b -->

In [1]:
from langchain_core.prompts.chat import AIMessagePromptTemplate
from langchain_core.prompts.chat import HumanMessagePromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama

In [ ]:
SystemMessage = SystemMessagePromptTemplate.from_template(template='You are a intelligent bot')
HumanMessage = HumanMessagePromptTemplate.from_template(template = "what comes after A")
AIMessage = AIMessagePromptTemplate.from_template(template = "It's B")
HumanMessage1 = HumanMessagePromptTemplate.from_template(template = "what comes after C")


template = ChatPromptTemplate.from_messages([SystemMessage, HumanMessage, AIMessage, HumanMessage1])

chat = ChatOllama(
    model = "llama3.2:1b",
    temperature = 0.8,
    num_predict = 200
)
# response = chat.invoke(template.format(salary = "30000"))
response = chat.invoke(template.format())
print(response.content)

A clever pattern to follow.

The sequence appears to be "a" followed by "b", and then it repeats. So, if we continue:

B (what comes after A) would be D.
D (what comes after B) would be E.
E (what comes after C) would be F.

Therefore, the correct sequence is: A, B, D, E, F


In [3]:
model = OllamaLLM(model="llama3.2:1b")
model.invoke("Come up with 10 names for a song about parrots")

'Here are ten potential song title ideas about parrots:\n\n1. "Rainbow Wings"\n2. "Paradise Found"\n3. "Squawk Sanctuary"\n4. "The Feathered Flock"\n5. "Colorful Chaos"\n6. "Whistle While You Work" (a play on the parrot\'s signature whistling)\n7. "In the Palm of Your Hand"\n8. "Tropical Temptation"\n9. "The Parrot\'s Perch"\n10. "Flock Together"'

In [4]:
from langchain_core.output_parsers.list import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

In [5]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [10]:
from langchain_core.caches import InMemoryCache
# Initialize cache
cache = InMemoryCache()

In [15]:
SystemMessage = SystemMessagePromptTemplate.from_template(template=output_parser.get_format_instructions())
HumanMessage = HumanMessagePromptTemplate.from_template(template = "Just Give me five characteristics of {animal}.")

template = ChatPromptTemplate.from_messages([SystemMessage, HumanMessage])
chat = ChatOllama(
    model = "llama3.2:1b",
    temperature = 0.8,
    num_predict = 200,
    cache = cache
)
print(output_parser.parse(chat.invoke(template.format(animal = "lion")).content))

['Here are five characteristics of lions:', 'Panthera leo', 'carnivorous mammal', 'social animal', 'nocturnal', 'apex predator']


In [17]:
from langchain.output_parsers.datetime import DatetimeOutputParser
dateTimeParser = DatetimeOutputParser()

In [18]:
dateTimeParser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1580-04-29T23:50:31.613388Z, 0666-01-05T17:37:56.325966Z, 1956-05-14T19:26:09.056534Z\n\nReturn ONLY this string, no other words!"

In [69]:
SystemMessage = SystemMessagePromptTemplate.from_template(template = "Always provide the output in date only.")
HumanMessage = HumanMessagePromptTemplate.from_template(template = "{request} {format}")
template = ChatPromptTemplate.from_messages([SystemMessage, HumanMessage])

chat = ChatOllama(
    model = "llama3.2:1b",
    temperature = 0,
    num_predict = 200,
    cache = cache
)

print(dateTimeParser.parse(chat.invoke(template.format(request = "TajMahal is built on which date?", format = dateTimeParser.get_format_instructions())).content))
# misformatted = dateTimeParser.parse(chat.invoke(template.format(request = "TajMahal is built on which date?", format = dateTimeParser.get_format_instructions())).content)


2022-04-11 14:30:00


In [70]:
# from langchain.output_parsers.fix import OutputFixingParser
# parser = OutputFixingParser.from_llm(llm = chat, parser= dateTimeParser)
# parser.parse(misformatted)

In [71]:
!pip install pydantic


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Creating custom Parsers

In [76]:
from langchain_core.output_parsers.pydantic import PydanticOutputParser
from pydantic import BaseModel, Field

In [77]:
class Scientist(BaseModel):
    name : str = Field(description="Name of the Scientist")
    discoveries : list = Field(description="Python list of discoveries")

In [79]:
parser = PydanticOutputParser(pydantic_object = Scientist)
print(parser.get_format_instructions() )

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of the Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}
```


In [85]:
# SystemMessage = SystemMessagePromptTemplate.from_template(template = parser.get_format_instructions())
HumanMessage = HumanMessagePromptTemplate.from_template(template = "{request} {format}")

template = ChatPromptTemplate.from_messages([HumanMessage])

chat = ChatOllama(
    model = "llama3.2:1b",
    temperature = 0,
    num_predict = 200,
    cache = cache
)

print(chat.invoke(template.format(request = "Tell me about famous scientist", format = parser.get_format_instructions())).content)


Here is a JSON instance that conforms to the provided schema:

```
{
  "properties": {
    "name": {
      "description": "Name of the Scientist",
      "title": "Name",
      "type": "string"
    },
    "discoveries": {
      "description": "Python list of discoveries",
      "items": {},
      "title": "Discoveries",
      "type": "array"
    }
  },
  "required": ["name", "discoveries"]
}
```

This object has two properties: `name` and `discoveries`. The `name` property is a string, and the `discoveries` property is an array of strings. Both are required.


# Serializing
## Saving and Loading the prompts

In [89]:
from langchain_core.prompts.prompt import PromptTemplate
prompt =  PromptTemplate(template = "Tell me a {anything}", input_variables = ['anything'])
print(prompt)
prompt.format(anything = "story")

input_variables=['anything'] input_types={} partial_variables={} template='Tell me a {anything}'


'Tell me a story'

In [90]:
prompt.save('prompt.json')

In [95]:
from langchain_core.prompts.loading import load_prompt
loaded_promt = load_prompt(path='prompt.json')
print(loaded_promt)

input_variables=['anything'] input_types={} partial_variables={} template='Tell me a {anything}'
